# Mock AlphaGo Zero (3B) Reinforced Learning
In this notebook, we will train the policy network by letting them compete each other according to DeepMind:

In [1]:
import os, numpy as np
from caffe2.python import core, model_helper, workspace, brew, utils
from caffe2.proto import caffe2_pb2
from sgfutil import BOARD_POSITION

%matplotlib inline
from matplotlib import pyplot

# how many games will be run in one minibatch
GAMES_BATCHES = 16 # [1,infinity) depends on your hardware
SEARCH_WIDE = 1600 # [1, infinity) for each step, run MCTS to obtain better distribution
# how many iterations for this tournament
TOURNAMENT_ITERS = 1 # [1,infinity)

if workspace.has_gpu_support:
    device_opts = core.DeviceOption(caffe2_pb2.CUDA, workspace.GetDefaultGPUID())
    print('Running in GPU mode on default device {}'.format(workspace.GetDefaultGPUID()))
else :
    device_opts = core.DeviceOption(caffe2_pb2.CPU, 0)
    print('Running in CPU mode')

arg_scope = {"order": "NCHW"}

ROOT_FOLDER = os.path.join(os.path.expanduser('~'), 'python', 'tutorial_data','zero','param') # folder stores the loss/accuracy log

Running in CPU mode


In [2]:
# Only 3 features are needed for AlphaGo Zero
# 0 - Player Stone, 1 - Opponent Stone, 3 - Current Player Color
DEFAULT_FEATURES = ["board", "color"]

# reverse the index of player/opponent
# 0,2,4,6... are player, 1,3,5,7... are opponent
OPPONENT_INDEX = [1,0,3,2,5,4,7,6,9,8,11,10,13,12]

We need to differentiate primary player and sparring partner. Primary player will learn from the game result

In [3]:
### Config for primary player
PRIMARY_WORKSPACE = os.path.join(ROOT_FOLDER, 'primary')
PRIMARY_RES_BLOCKS = 1 # [1,19(AlphaGo Zero),39]
PRIMARY_FILTERS = 128 # [128, 192, 256(AlphaGo Zero), 384]
PRIMARY_PRE_TRAINED_ITERS = 0
# before traning, where to load the params
PRIMARY_LOAD_FOLDER = os.path.join(ROOT_FOLDER, "RL-res={}-k={}-iter={}"
                                   .format(PRIMARY_RES_BLOCKS,PRIMARY_FILTERS,PRIMARY_PRE_TRAINED_ITERS))
BASE_LR = -0.01 # (-0.01,0) The base Learning Rate; 0 to disable it.
TRAIN_BATCHES = 16 # how many samples will be trained within one mini-batch, depends on your hardware
# after training, where to store the params
PRIMARY_SAVE_FOLDER = os.path.join(ROOT_FOLDER, "RL-res={}-k={}-iter={}"
                           .format(PRIMARY_RES_BLOCKS,PRIMARY_FILTERS,PRIMARY_PRE_TRAINED_ITERS+TOURNAMENT_ITERS))
if not os.path.exists(PRIMARY_SAVE_FOLDER):
    os.makedirs(PRIMARY_SAVE_FOLDER)

### Config for sparring partner
SPARR_WORKSPACE = os.path.join(ROOT_FOLDER, 'sparring')
SPARR_LOAD_FOLDER = os.path.join(ROOT_FOLDER, "res={}-k={}-iter={}".format(1,128,1))

print('Training model from {} to {} iterations'.format(PRIMARY_PRE_TRAINED_ITERS,PRIMARY_PRE_TRAINED_ITERS+TOURNAMENT_ITERS))

Training model from 0 to 1 iterations


## AlphaGo Neural Network Architecture

In [4]:
from modelingZero import AddResNetModel, AddSoftmax, AddTrainingOperators

## Build the actual network

In [5]:
import caffe2.python.predictor.predictor_exporter as pe

data = np.empty(shape=(TRAIN_BATCHES,17,19,19), dtype=np.float32)
expect = np.empty(shape=(TRAIN_BATCHES,362), dtype=np.float32) # expected distribution of probability
reward = np.empty(shape=(TRAIN_BATCHES,), dtype=np.float32) # scalar values between [-1,1]

### Primary player
>Train Net: Blob('data','label') ==> Predict Net ==> Loss ==> Backward Propergation

In [6]:
workspace.SwitchWorkspace(PRIMARY_WORKSPACE, True)

with core.DeviceScope(device_opts):
    workspace.FeedBlob("data", data)
    workspace.FeedBlob('expect', expect)
    workspace.FeedBlob('reward', reward)
    # for learning from winner
    primary_train_model = model_helper.ModelHelper(name="primary_train_model", arg_scope=arg_scope, init_params=True)
    predict, value = AddResNetModel(primary_train_model, 'data', num_blocks=PRIMARY_RES_BLOCKS, filters=PRIMARY_FILTERS)
    AddTrainingOperators(primary_train_model, predict, None, 'expect', value, 'reward', base_lr=BASE_LR)
    workspace.RunNetOnce(primary_train_model.param_init_net)
    workspace.CreateNet(primary_train_model.net, overwrite=True)
    # 
    primary_predict_net = pe.prepare_prediction_net(os.path.join(PRIMARY_LOAD_FOLDER, "policy_model.minidb"), "minidb")

Function `LearnFromWinner` takes the result of tournament and train primary player with the result.

In [7]:
def LearnFromWinningGames(history, winner, mini_batch=TRAIN_BATCHES):
    data = np.empty(shape=(mini_batch,17,19,19), dtype=np.float32)
    label = np.empty(shape=(mini_batch,), dtype=np.int32)
    #iter = 0
    k = 0
    for i in range(len(winner)):
        #print('Learning {} steps in {} of {} games'.format(iter * TRAIN_BATCHES, i, GAMES_BATCHES))
        for step in history[i]:
            if (step[0] == 'B' and winner[i] == 'B+') or (step[0] == 'W' and winner[i] == 'W+'):
                data[k] = step[2]
                label[k] = step[1]
                k += 1
                #iter += 1
                if k == mini_batch:
                    k = 0
                    workspace.SwitchWorkspace(PRIMARY_WORKSPACE)
                    with core.DeviceScope(device_opts):
                        workspace.FeedBlob("data", data)
                        workspace.FeedBlob("label", label)
                        workspace.RunNet(primary_train_model.net)

### Sparring partner
    Load on the fly

## Run the tournament and training
>We use a reward function $r(s)$ that is zero for all non-terminal time-steps $t < T$.
The outcome $z_t = \pm r(s_T)$ is the terminal reward at the end of the game from the perspective of the
current player at time-step $t$: $+1$ for winning and $-1$ for losing. Weights are then updated at each
time-step $t$ by stochastic gradient ascent in the direction that maximizes expected outcome.

In [8]:
from go import GameState, BLACK, WHITE, EMPTY, PASS
from preprocessing import Preprocess
from datetime import datetime
from sgfutil import GetWinner, WriteBackSGF
import sgf

np.random.seed(datetime.now().microsecond)

# construct the model to be exported
pe_meta = pe.PredictorExportMeta(
    predict_net=primary_predict_net.Proto(),
    parameters=[str(b) for b in primary_train_model.params],
    inputs=["data"],
    outputs=["softmax"],
)

In [9]:
for tournament in range(PRIMARY_PRE_TRAINED_ITERS, PRIMARY_PRE_TRAINED_ITERS+TOURNAMENT_ITERS):
    # Every 500 tournament, copy current player to opponent. i.e. checkpoint
    if tournament > 0 and tournament % 20 == 0:
        pe.save_to_db("minidb", os.path.join(PRIMARY_SAVE_FOLDER, "policy_model.minidb"), pe_meta)
        print('Checkpoint saved to {}'.format(PRIMARY_SAVE_FOLDER))
        pe.save_to_db("minidb", os.path.join(SPARR_LOAD_FOLDER, "policy_model_RL_{}.minidb".format(PRIMARY_PRE_TRAINED_ITERS+tournament)), pe_meta)
        print('Checkpoint saved to {}'.format(SPARR_LOAD_FOLDER))
    
    # Randomly change color of player
    PRIMARY_PLAYER = np.random.choice(['B','W'])
    if PRIMARY_PLAYER == 'B':
        SPARRING_PLAYER = 'W'
    else:
        SPARRING_PLAYER = 'B'
    
    # Randomly pickup sparring partner
    workspace.SwitchWorkspace(SPARR_WORKSPACE, True)
    sparring_param_file = np.random.choice(os.listdir(SPARR_LOAD_FOLDER))
    with core.DeviceScope(device_opts):
        sparring_predict_net = pe.prepare_prediction_net(os.path.join(SPARR_LOAD_FOLDER, sparring_param_file), "minidb")
    print('Tournament {} Primary({}) vs Sparring({}|{}) started @{}'
          .format(tournament, PRIMARY_PLAYER, SPARRING_PLAYER, sparring_param_file, datetime.now()))

    
    # Initialize game board and game state
    game_state = [ GameState() for i in range(GAMES_BATCHES) ]
    game_result = [0] * GAMES_BATCHES # 0 - Not Ended; BLACK - Black Wins; WHITE - White Wins
    p = Preprocess(DEFAULT_FEATURES) # Singleton
    history = [ [] for i in range(GAMES_BATCHES) ] # history[n][step] stores tuple of (player, x, y, board[n])
    board = None # The preprocessed board with shape Nx17x19x19
    
    # for each step in all games
    for step in range(0,722):
        
        # Preprocess the board
        board = np.concatenate([p.state_to_tensor(game_state[i]).astype(np.float32) for i in range(GAMES_BATCHES)])

        if step % 2 == 0:
            current_player = BLACK
            current_color = 'B'
        else:
            current_player = WHITE
            current_color = 'W'

        if step % 2 == (PRIMARY_PLAYER == 'W'): # if step %2 == 0 and Primary is Black, or vice versa.
            # primary player make move
            workspace.SwitchWorkspace(PRIMARY_WORKSPACE)
            with core.DeviceScope(device_opts):
                workspace.FeedBlob('data', board)
                workspace.RunNet(primary_predict_net)
        else:
            # sparring partner make move
            workspace.SwitchWorkspace(SPARR_WORKSPACE)
            with core.DeviceScope(device_opts):
                workspace.FeedBlob('data', board)
                workspace.RunNet(sparring_predict_net)

        predict = workspace.FetchBlob('softmax') # [0.01, 0.02, ...] in shape (N,361)

        for i in range(GAMES_BATCHES):
            if game_result[i]: # game end
                continue
            else: # game not end
                legal_moves = [ x*19+y for (x,y) in game_state[i].get_legal_moves(include_eyes=False)] # [59, 72, ...] in 1D
                if len(legal_moves) > 0: # at least 1 legal move
                    probabilities = predict[i][legal_moves] # [0.02, 0.01, ...]
                    # use numpy.random.choice to randomize the step,
                    # otherwise use np.argmax to get best choice
                    # current_choice = legal_moves[np.argmax(probabilities)]
                    if np.sum(probabilities) > 0:
                        current_choice = np.random.choice(legal_moves, 1, p=probabilities/np.sum(probabilities))[0]
                    else:
                        current_choice = np.random.choice(legal_moves, 1)[0]
                    (x, y) = (current_choice/19, current_choice%19)
                    history[i].append((current_color, current_choice, board[i]))
                    game_state[i].do_move(action = (x, y), color = current_player) # End of Game?
                    #print('game({}) step({}) {} move({},{})'.format(i, step, current_color, x, y))
                else:
                    game_state[i].do_move(action = PASS, color = current_player)
                    #print('game({}) step({}) {} PASS'.format(i, step, current_color))
                    game_result[i] = game_state[i].is_end_of_game

        if np.all(game_result):
            break
    
    # Get the winner
    winner = [ GetWinner(game_state[i]) for i in range(GAMES_BATCHES) ] # B+, W+, T
    print('Tournament {} Finished with Primary({}) {}:{} Sparring({}) @{}'.
          format(tournament, PRIMARY_PLAYER, sum(np.char.count(winner, PRIMARY_PLAYER)),
                 sum(np.char.count(winner, SPARRING_PLAYER)), SPARRING_PLAYER, datetime.now()))
    
    # Save the games(optional)
    for i in range(GAMES_BATCHES):
        filename = os.path.join(
            os.path.expanduser('~'), 'python', 'tutorial_files','selfplay',
            '({}_{}_{})vs({})_{}_{}_{}'.format(PRIMARY_CONV_LEVEL, PRIMARY_FILTERS, PRIMARY_PRE_TRAINED_ITERS+tournament,
                                            sparring_param_file, i, winner[i],
                                            datetime.now().strftime("%Y-%m-%dT%H:%M:%S%Z")))
        WriteBackSGF(winner, history[i], filename)
    
    # After each tournament, learn from the winner
    if BASE_LR != 0:
        LearnFromWinningGames(history, winner, mini_batch=TRAIN_BATCHES)

Tournament 0 Primary(B) vs Sparring(W|policy_model.minidb) started @2017-11-01 18:30:04.795610
Traceback for operator 1 in network policy_deploy_1


RuntimeError: [enforce fail at conv_op_impl.h:46] C == filter.dim32(1) * group_. Convolution op: input channels does not match: # of input channels 4 is not equal to kernel channels * group:17*1 Error from operator: 
input: "pad1" input: "conv1_w" input: "conv1_b" output: "conv1" name: "" type: "Conv" arg { name: "kernel" i: 3 } arg { name: "exhaustive_search" i: 0 } arg { name: "order" s: "NCHW" } device_option { device_type: 0 cuda_gpu_id: 0 } engine: "CUDNN"

In [ ]:
if TOURNAMENT_ITERS>0 :
    pe.save_to_db("minidb", os.path.join(PRIMARY_SAVE_FOLDER, "policy_model.minidb"), pe_meta)
    print('Results saved to {}'.format(PRIMARY_SAVE_FOLDER))
    pe.save_to_db("minidb", os.path.join(SPARR_LOAD_FOLDER, "policy_model_RL_{}.minidb".format(PRIMARY_PRE_TRAINED_ITERS+TOURNAMENT_ITERS)), pe_meta)
    print('Results saved to {}'.format(SPARR_LOAD_FOLDER))